# Training Rete Neurale per progetto Sistemi Digitali A.S. 2021/2022


# Install all the needed dependencies

In [1]:
!pip uninstall --yes tflite-model-maker && yes | pip install tflite-model-maker==0.3.4
!pip uninstall tensorflow-tensorboard && pip install tensorboard
!pip install pycocotools
!pip install tflite-support
!pip list

from google.colab import drive
from google.colab import files
from datetime import datetime
from PIL import Image
import pathlib
import threading
import numpy as np
import tensorflow as tf
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tflite_model_maker.object_detector import DataLoader
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import train
from tensorflow_examples.lite.model_maker.third_party.efficientdet.keras import train_lib
from tensorflow_examples.lite.model_maker.core.task import model_util

%load_ext tensorboard

assert tf.__version__.startswith('2')

     |████████████████████████████████| 616 kB 5.3 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 120 kB 47.4 MB/s 
     |████████████████████████████████| 6.4 MB 30.0 MB/s 
     |████████████████████████████████| 237 kB 38.8 MB/s 
     |████████████████████████████████| 1.1 MB 21.5 MB/s 
     |████████████████████████████████| 3.4 MB 29.1 MB/s 
     |████████████████████████████████| 840 kB 16.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 27.8 MB/s 
     |████████████████████████████████| 77 kB 5.3 MB/s 
     |████████████████████████████████| 596 kB 35.1 MB/s 
     |████████████████████████████████| 25.3 MB 6.6 MB/s 
     |████████████████████████████████| 352 kB 39.0 MB/s 
     |████████████████████████████████| 47.8 MB 64 kB/s 
     |████████████████████████████████| 99 kB 5.2 MB/s 
     |████████████████████████████████| 462 kB 33.1 MB/s 
     |████████████████

# Import the images into google colab and creates all the files later required

In [2]:
force_dataset_creation = True
download_original_dataset = False
run_data_augmentation = False
resize_needed = False
image_size = 320 #used for resizing only
data_dir = "/content/dataset/"
data_path = pathlib.Path(data_dir)
thread_list = []




def get_subject_images_paths(subject_number):
  path = "/content/dataset/Subject\ {}".format(str(subject_number))
  images_paths = !find $path -type f
  return images_paths

def get_subjects_images_paths(low_subject_number, high_subject_number):
  images_paths = []
  for subject_number in range(low_subject_number, high_subject_number + 1):
    images_paths = np.append(images_paths, get_subject_images_paths(subject_number)).flatten()
  return images_paths

def unzip(original_path, final_path):
  !unzip -u $original_path -d $final_path & #for zip files

def resize(path, new_size):
  print(path)
  !convert '$path' -resize "$new_size"x"$new_size" '$path'
  
def resize_in_parallel(subject_number, new_size):
  paths = get_subject_images_paths(subject_number)
  for path in paths:
    resize(path, new_size)





if force_dataset_creation == True or data_path.exists() == False:
  #Removes the dataset from content directory
  !rm -r "/content/dataset/"
  !rm -r "/content/data-augmentation/"

  #mounts google drive to access uploaded files/folders 
  drive.mount('/content/drive/', force_remount = True)

  dataset_to_download = "Original" if download_original_dataset == True else "Augmented"

  #extracts all the images and labels from their zips in parallel 
  t1 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/WWMR-DB-Part\ 1-" + dataset_to_download + ".zip", "/content/dataset/",))
  t2 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/WWMR-DB-Part\ 2-" + dataset_to_download + ".zip", "/content/dataset/",))
  t3 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/WWMR-DB-Part\ 3-" + dataset_to_download + ".zip", "/content/dataset/",))
  t4 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/WWMR-DB-Part\ 4-" + dataset_to_download + ".zip", "/content/dataset/",))
  t5 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/WWMR-DB-Labels-"  + dataset_to_download + ".zip", "/content/dataset/",))
  t6 = threading.Thread(target = unzip, args = ("/content/drive/MyDrive/Colab\ Notebooks/images_zips/data-augmentation-cpu.zip",                       "/content/",))

  thread_list.append(t1)
  thread_list.append(t2)
  thread_list.append(t3)
  thread_list.append(t4)
  thread_list.append(t5)
  thread_list.append(t6)
  t1.start()
  t2.start()
  t3.start()
  t4.start()
  t5.start()
  t6.start()

  for tr in thread_list:
    tr.join()
    print("Done")



if run_data_augmentation == True:
    !python "/content/data-augmentation/main.py"
    #!zip -r "/content/dataset.zip" "/content/dataset/"




if resize_needed == True:
  #Install the needed dependencies
  !sudo apt update
  !sudo apt install imagemagick
  
  #Resize all subjects' images
  thread_list = []
  for subject_number in range(1,43):
    t = threading.Thread(target = resize_in_parallel, args = (subject_number, image_size,))
    thread_list.append(t)
    t.start()

  for t in thread_list:
    t.join()
    print("Resized")

  #Compress the new resized dataset in a zip file
  !zip -r "/content/dataset.zip" "/content/dataset/"

Output streaming troncato alle ultime 5000 righe.
  inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_ROTATE_N31.txt  
  inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_ROTATE_P156.txt  
  inflating: /content/dataset/Subject 20/Mask Or Respirator Under The Chin/Surgical Mask/0020_MRNC_SRGM_0045_NOISE.jpg    inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_SCALE.txt  
  inflating: /content/dataset/Subject 40/Mask Or Respirator Under The Nose/Non-Medical Mask/0040_MRNN_NMDM_0000_BLUR.jpg  
  inflating: /content/dataset/Subject 5/Mask Or Respirator Hanging From An Ear/Non-Medical Mask/0005_MRHN_NMDM_0045_B_TRASLT.jpg    inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_SHARP.txt  
  inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_TRASLT.txt  
  inflating: /content/dataset/Labels/YOLO/Subject 41/0041_MRHN_SRGM_0090_F_VFLIP.txt  
  inflating: /content/dataset/Labels/YOLO/S

# Create the dataset csv file

In [3]:
%tensorflow_version 2.x

def get_subject_images_paths(subject_number):
  path = "/content/dataset/Subject\ {}".format(str(subject_number))
  images_paths = !find $path -type f
  return images_paths

def get_subjects_images_paths(low_subject_number, high_subject_number):
  images_paths = []
  for subject_number in range(low_subject_number, high_subject_number + 1):
    images_paths.extend(get_subject_images_paths(subject_number)) 
  return images_paths

def get_bouding_client_rect(subject_number, filename, label):
  bounding_rects_file = "/content/dataset/Labels/YOLO/Subject {}/{}.txt".format(subject_number, filename)
  mask_labels = ["MRNW"] #["SRGM", "DRWV", "DRNV", "NMDM", "MRNW"]
  
  with open(bounding_rects_file, "r") as file:
    if label not in mask_labels:   
      file.readline()   
      file.readline()

    line = file.readline()
    line_parts = line.split(" ")[1:]
    xmin = float(line_parts[0]) - 0.5 * float(line_parts[2])
    ymin = float(line_parts[1]) - 0.5 * float(line_parts[3])
    xmax = float(line_parts[0]) + 0.5 * float(line_parts[2])
    ymax = float(line_parts[1]) + 0.5 * float(line_parts[3])    

  return "{:.7f}".format(xmin), "{:.7f}".format(ymin), "{:.7f}".format(xmax), "{:.7f}".format(ymax)

def get_subject_number(filename_without_ext):
  return int(filename_without_ext.split("_")[0])

def get_filename_without_ext(path):
  return path.split("/")[-1].split(".")[0]

def get_labels(filename_without_ext):
  label_parts = filename_without_ext.split("_")[1:-1]
  labels = [label_parts[0]] 
  if labels[0] != "MRNW":
    labels.append(label_parts[1])
  return labels

def concat_labels(labels):
  final_label = ""
  for label in labels:
    final_label += label + "_"
  return final_label[:-1]

def get_prefix(test_num, val_num, subject_num):
  global test_index, val_index, train_index, last_val_subject_num
  
  prefix_selector = (test_index + val_index) % 2
  
  if prefix_selector == 0 and test_index < test_num:
    test_index += 1
    return "TEST"
  elif val_index < val_num or last_val_subject_num == subject_num:
    val_index += 1
    last_val_subject_num = subject_num
    return "VALIDATE"
  
  return "TRAIN"




force_csv_creation = True
paths = get_subjects_images_paths(1, 42)
test_num = 50
val_num = 0.2 * (len(paths) - test_num)

csv_dataset_dir = "/content/dataset/dataset_object_detector.csv"
test_index = 0
val_index = 0
last_val_subject_num = 1;

#https://cloud.google.com/vision/automl/object-detection/docs/csv-format
#prefix, path, label, xmin, ymin, xmax, ymin, xmax, ymax, xmin, ymax
#prefix = TEST | TRAIN | VALIDATE
if force_csv_creation == True or pathlib.Path(csv_dataset_dir).exists() == False:
  with open(csv_dataset_dir, "w", encoding="utf-8") as file:
    for path in paths:
      filename = get_filename_without_ext(path)
      subject_number = get_subject_number(filename)
      prefix = get_prefix(test_num, val_num, subject_number)
      labels = get_labels(filename)
      final_label = concat_labels(labels)

      xmin, ymin, xmax, ymax = get_bouding_client_rect(subject_number, filename, labels[0])
      file.write(prefix + "," + path + "," + final_label + "," + xmin + "," + ymin + "," + xmax + "," + ymin + "," + xmax + "," + ymax + "," + xmin + "," + ymax + "\n")

# Create and train the object detector

In [4]:
%tensorflow_version 2.x

train_before_exporting_model = True
resume_from_last_checkpoint = True

#Available specs at:
#https://www.tensorflow.org/lite/tutorials/model_maker_object_detection
spec_heavy = True
spec_light = not spec_heavy   
tf_model_dir = "model_tf/"
dataset_dir = "/content/dataset/dataset_object_detector.csv"
tensorboard_dev_name = "exp_" + datetime.now().strftime("%Y%m%d-%H%M%S")

batch_size = 7 #it takes 50min for an epoch with batch_size = 6
completed_epochs = 42 
training_epochs = completed_epochs + 6
tflite_filename = "model_heavy.tflite"
checkpoint_dir = "/content/drive/My Drive/Colab Notebooks/checkpoints_heavy/"
checkpoint_dir_escaped = "/content/drive/My\ Drive/Colab\ Notebooks/checkpoints_heavy/"
export_dir = "/content/drive/My Drive/Colab Notebooks/model_heavy/"
tensorboard_init_dir = "/content/drive/My\ Drive/Colab\ Notebooks/model_heavy/tensorboard_logs/" 
model_name = "efficientdet-lite4"
model_uri = "https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2"
spec = object_detector.EfficientDetLite4Spec(
    model_name = model_name,
    uri = model_uri, 
    hparams='', 
    model_dir=checkpoint_dir, 
    epochs=training_epochs, 
    batch_size=batch_size,
    steps_per_execution=1, 
    moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=10, 
    strategy=None, 
    tpu=None, 
    gcp_project=None,
    tpu_zone=None, 
    use_xla=False, 
    profile=False, 
    debug=False, 
    tf_random_seed=111111,
    verbose=1
)

if spec_light == True:
  batch_size = 50
  training_epochs = 50
  tflite_filename = "model_light.tflite"
  checkpoint_dir = "/content/drive/My Drive/Colab Notebooks/checkpoints_light/"
  checkpoint_dir_escaped = "/content/drive/My\ Drive/Colab\ Notebooks/checkpoints_light/"
  export_dir = "/content/drive/My Drive/Colab Notebooks/model_light/"
  tensorboard_init_dir = "/content/drive/My\ Drive/Colab\ Notebooks/model_light/tensorboard_logs/" 
  model_name = "efficientdet-lite0"
  model_uri = "https://tfhub.dev/tensorflow/efficientdet/lite0/feature-vector/1"
  spec = object_detector.EfficientDetLite0Spec(
    model_name = model_name,
    uri = model_uri, 
    hparams='', 
    model_dir=checkpoint_dir, 
    epochs=training_epochs, 
    batch_size=batch_size,
    steps_per_execution=1, 
    moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=25, 
    strategy=None, 
    tpu=None, 
    gcp_project=None,
    tpu_zone=None, 
    use_xla=False, 
    profile=False, 
    debug=False, 
    tf_random_seed=111111,
    verbose=1
  )

tensorboard_dir =  export_dir + "tensorboard_logs/" #+ datetime.now().strftime("%Y%m%d-%H%M%S")

#https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector
#https://github.com/tensorflow/examples/blob/189c662d500c20b9f9e93fef0af97d5311e64377/tensorflow_examples/lite/model_maker/core/task/object_detector.py#L92
#https://github.com/tensorflow/examples/blob/189c662d500c20b9f9e93fef0af97d5311e64377/tensorflow_examples/lite/model_maker/core/task/model_spec/object_detector_spec.py#L241
#https://github.com/tensorflow/examples/blob/189c662d500c20b9f9e93fef0af97d5311e64377/tensorflow_examples/lite/model_maker/third_party/efficientdet/keras/train.py#L111
#https://github.com/tensorflow/examples/blob/189c662d500c20b9f9e93fef0af97d5311e64377/tensorflow_examples/lite/model_maker/core/task/custom_model.py
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(dataset_dir)
detector = object_detector.create(train_data, 
                                  model_spec=spec, 
                                  batch_size=batch_size, 
                                  train_whole_model=True, 
                                  validation_data=validation_data,
                                  epochs = training_epochs,
                                  do_train = False
                                 )

#Convert the datasets for training
train_ds, steps_per_epoch, _ = detector._get_dataset_and_steps(train_data, batch_size, is_training = True)
validation_ds, validation_steps, val_json_file = detector._get_dataset_and_steps(validation_data, batch_size, is_training = False)

#Create and compile the keras model using internal tflite model maker APIs     
model = detector.create_model()
config = spec.config
config.update(
    dict(
        steps_per_epoch=steps_per_epoch,
        eval_samples=batch_size * validation_steps,
        val_json_file=val_json_file,
        batch_size=batch_size
    )
)
train.setup_model(model, config)
model.summary()

#Load the previously saved model's weights if there's any 
try:
  if resume_from_last_checkpoint == False:
    !rm -r $checkpoint_dir_escaped
    !rm -r $tensorboard_init_dir
    !mkdir $checkpoint_dir_escaped
    !mkdir $tensorboard_init_dir

  #latest = tf.train.latest_checkpoint(checkpoint_dir)
  #model.load_weights(latest)
  #print("\nCheckpoint found {}\n".format(latest)) 
  tf.train.load_checkpoint("/content/drive/My Drive/Colab Notebooks/checkpoints_heavy/ckpt-35")
  print("\nCheckpoint found {}\n".format("35")) 
except Exception as e:
  print("\nCheckpoint not found\n")

#Create custom epoch's callbacks and train the model 
if train_before_exporting_model == True:
  #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=tensorboard_dir, histogram_freq=1)
  callbacks = train_lib.get_callbacks(config.as_dict(), validation_ds)
  #callbacks.append(tensorboard_callback)

  history = model.fit(
      train_ds,
      epochs=training_epochs,
      initial_epoch=completed_epochs,
      steps_per_epoch=steps_per_epoch,
      validation_data=validation_ds,
      validation_steps=validation_steps,
      callbacks=callbacks  
  )

#Inject the trained model into the tflite object detector object
#so that the API's exporting methods can be used 
detector.model = model

#detector.evaluate_tflite(export_dir + tflite_filename, test_data)

INFO:tensorflow:Cache will be stored in /tmp/tmpsi6x2_ts with prefix filename train_0849233ad97e5b32bee3940056cd4d67. Cache_prefix is /tmp/tmpsi6x2_ts/train_0849233ad97e5b32bee3940056cd4d67
INFO:tensorflow:Cache will be stored in /tmp/tmphtd8uu1b with prefix filename val_0849233ad97e5b32bee3940056cd4d67. Cache_prefix is /tmp/tmphtd8uu1b/val_0849233ad97e5b32bee3940056cd4d67
INFO:tensorflow:Cache will be stored in /tmp/tmp8o9edf62 with prefix filename test_0849233ad97e5b32bee3940056cd4d67. Cache_prefix is /tmp/tmp8o9edf62/test_0849233ad97e5b32bee3940056cd4d67
INFO:tensorflow:On image 0
INFO:tensorflow:On image 100
INFO:tensorflow:On image 200
INFO:tensorflow:On image 300
INFO:tensorflow:On image 400
INFO:tensorflow:On image 500
INFO:tensorflow:On image 600
INFO:tensorflow:On image 700
INFO:tensorflow:On image 800
INFO:tensorflow:On image 900
INFO:tensorflow:On image 1000
INFO:tensorflow:On image 1100
INFO:tensorflow:On image 1200
INFO:tensorflow:On image 1300
INFO:tensorflow:On image 140

ResourceExhaustedError: ignored

# Quantize the trained model and converts it to .tflite format  

In [ ]:
detector.model = model

#Convert the model to a .tflite format using full integer quantization (default quantization)
detector.export(export_dir = export_dir, tflite_filename = "int_8_" + tflite_filename) 

#Convert the model to a .tflite format using float 16 quantization
quantization_config=QuantizationConfig.for_float16()
detector.export(export_dir = export_dir, tflite_filename = "float_16_" + tflite_filename, quantization_config=quantization_config) 
detector.evaluate_tflite(export_dir + "float_16_" + tflite_filename, test_data)

INFO:tensorflow:Assets written to: /tmp/tmpctax6xac/assets


INFO:tensorflow:Assets written to: /tmp/tmpctax6xac/assets


# Upload the training logs to tensorboard.dev

In [ ]:
#https://tensorboard.dev/experiments/
#%tensorboard --logdir tensorboard_init_dir

!tensorboard dev upload --logdir {tensorboard_init_dir} --name tensorboard_dev_name --one_shot

#Old way

Creates and train the model with the imported images

In [ ]:
%tensorflow_version 2.x
from google.colab import drive
import os
import pathlib
import tensorflow as tf
from tensorflow import keras
from tensorflow.lite.python import optimize
from tensorflow.python.ops.variables import trainable_variables
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# tf.compat.v1.enable_eager_execution()
# tf.config.run_functions_eagerly(True)
# tf.data.experimental.enable_debug_mode()

image_size = 160
number_of_subjects = 42
num_classes = 40 #12
train_ratio = 0.8 #80% of images used for training, 20% for validation

def get_subject_images_paths(subject_number):
  path = "/content/dataset/Subject\ {}".format(str(subject_number))
  images_paths = !find $path -type f
  return images_paths

def get_subjects_images_paths(low_subject_number, high_subject_number):
  images_paths = []
  for subject_number in range(low_subject_number, high_subject_number + 1):
    images_paths = np.append(images_paths, get_subject_images_paths(subject_number)).flatten()
  return images_paths

def get_label_as_tensor(filename):
  df = pd.read_csv("/content/dataset/dataset.csv", header = 0, delim_whitespace = False)
  all_labels = ["MSFC", "MRCW", "MRHN", "MRNW", "MRFH", "MRTN", "MRNC", "MRNN", "DRWV", "DRNV", "NMDM", "SRGM"]
  filename_row = ""
  for _filename in df["Filename"]:
    if tf.convert_to_tensor(_filename) == filename:
      filename_row = _filename
  labels = df[df["Filename"] == filename_row].to_numpy()[0][1:]
  return tf.convert_to_tensor(labels.astype(np.int64))

def load_images(image_path: tf.Tensor):
  global image_size
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels = 3)
  image = tf.image.convert_image_dtype(image, dtype = tf.float32)
  image = tf.image.resize(image, [image_size, image_size])

  #https://stackoverflow.com/questions/60997186/understanding-how-to-use-tf-dataset-map
  label = tf.numpy_function(
      func = get_label_as_tensor,
      inp = [image_path],
      Tout = tf.int64
  ) 

  return (image, label)

def augment_images(image, label):
  image = tf.image.random_flip_up_down(image)
  image = tf.image.random_flip_left_right(image)

  return image, label

paths = get_subjects_images_paths(1, number_of_subjects)
dataset = tf.data.Dataset.from_tensor_slices(paths)
dataset_size = dataset.cardinality().numpy()

train_ds = dataset.take(dataset_size * train_ratio)
val_ds = dataset.skip(dataset_size * train_ratio)

train_ds = (dataset
            .shuffle(len(dataset))
            .map(load_images, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
            .map(augment_images, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
            .cache()
            .batch(32)
            .prefetch(tf.data.AUTOTUNE)
           ) 

val_ds = (dataset
          .shuffle(len(dataset))
          .map(load_images, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
          .map(augment_images, num_parallel_calls = tf.data.AUTOTUNE, deterministic = False)
          .cache()
          .batch(32)
          .prefetch(tf.data.AUTOTUNE)
          ) 

#model creation
base_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape = (image_size, image_size, 3),
    include_top = False,
    weights = "imagenet"
)
base_model.trainable = False

######################################################################################################################### 
#bisogna far uscire 4 output per android studio
#potrebbe essere utile usare i file con le bounding boxes
#Bisogna fare un custom object detector e non solo un image classifier 
#https://stackoverflow.com/questions/44036971/multiple-outputs-in-keras
model = tf.keras.models.Sequential([ 
  #base_model,                                    
  tf.keras.layers.Rescaling(1./255, input_shape = (image_size , image_size , 3)),
  #Conv2D serve per togliere il rumore:
  #1) applica n filtri usando kernel diversi
  #2) relu toglie (setta a 0) tutti i valori (pixels) < 0 
  #in pratica toglie i pixel non rilevanti per l'n-esimo filtro applicato
  tf.keras.layers.Conv2D(16, 3, padding = "same", activation = "relu"),
  #MaxPool2D serve per prendere il valore massimo (pixel più rilevante) per ogni asse dell'immagine
  #(ogni 2 pixel prende il massimo) passata come input
  tf.keras.layers.MaxPool2D(padding = "same"),
  #Dato che potrebbe succedere di fare MaxPool2D tra 2 pixel che erano stati "tolti" (resi = 0) dalla Conv2D,
  #si riapplica la Conv2D e il MaxPool2D per cercare di avere meno "errori" possibile
  tf.keras.layers.Conv2D(32, 3, padding = "same", activation = "relu"),
  tf.keras.layers.MaxPool2D(padding = "same"),
  tf.keras.layers.Conv2D(64, 3, padding = "same", activation = "relu"),
  tf.keras.layers.MaxPool2D(padding = "same"),
  #Flatten "srotola" i tensori (array multidimensionali di pixel => immagini date dal livello sopra) in array lineari
  tf.keras.layers.Flatten(),
  #Dense applica la funzione activation ad ogni elemento in ingresso (ogni pixel)
  #relu toglie (setta a 0) tutti i valori (pixels) < 0
  #il primo numero indica quanti valori ci sono in uscita
  tf.keras.layers.Dense(128, activation = "relu"),
  tf.keras.layers.Dense(num_classes) 
])

print(model.summary())

#model compilation
optimizer_func = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
metric_func = tf.keras.metrics.CategoricalAccuracy() #perchè le nostre label sono one-hot encoded
model.compile(
  optimizer = optimizer_func,
  loss = loss_func,
  metrics = [metric_func],
  #run_eagerly = True
)

#create a callback that saves the model's weights
checkpoint_folder = "/content/checkpoints/"
checkpoint_path = checkpoint_folder + "checkpoint-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                         save_weights_only = True,
                                                         verbose = 1) 
#load weights from checkpoint if available
if pathlib.Path(checkpoint_folder).exists() == True:
  latest = tf.train.latest_checkpoint(checkpoint_dir)
  print("latest checkpoint found: " + latest)
  model.load_weights(latest)
else:
  print("No checkpoint found")
  
#model training
epochs = 1
history = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs = epochs,
  batch_size = dataset_size,
  #callbacks = [checkpoint_callback]
)

#plot metrics
# plt.plot(history.history['categorical_accuracy'])
# plt.show()

#save model
saved_model_dir = "/content/model/"
model.save(saved_model_dir)

Converts the model to a tensorflow lite model

In [ ]:
import tensorflow as tf

#convert the model to tflite
saved_model_dir = "/content/model/"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

#save the tflite model
with open("model.tflite", "wb") as f:
  f.write(tflite_model)

Writes the required metadata for the tensorflow lite model

In [ ]:
#!/usr/bin/env python3
!pip install tflite_support
import pathlib
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

#https://www.tensorflow.org/lite/convert/metadata

#creates a metadata writer
ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = "/content/model.tflite"

# Task Library expects label files that are in the same format as the one below.
_LABEL_FILE = "/content/model_labels.txt"
_SAVE_TO_PATH = "/content/model_metadata.tflite"

# Normalization parameters is required when reprocessing the image. It is
# optional if the image pixel values are in range of [0, 255] and the input
# tensor is quantized to uint8. See the introduction for normalization and
# quantization parameters below for more details.
# https://www.tensorflow.org/lite/convert/metadata#normalization_and_quantization_parameters)
_INPUT_NORM_MEAN = 127.5
_INPUT_NORM_STD = 127.5

# Create the metadata writer.
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), [_INPUT_NORM_MEAN], [_INPUT_NORM_STD],
    [_LABEL_FILE])


# Populate the metadata into the model.
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(_SAVE_TO_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())